Some calculations on nutriments

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Provision of further data

# Amino acid reference for a human(mg/g Protein), Source: ChatGPT (->WHO?)
aa_ref = {
  "Histidin": 15,
  "Isoleucin": 30,
  "Leucin": 59,
  "Lysin": 45,
  "Methionin + Cystein": 22,
  "Phenylalanin + Tyrosin": 38,
  "Threonin": 23,
  "Tryptophan": 6,
  "Valin": 39,

}

In [4]:
df = pd.read_excel("BLS_4_0_Daten_2025_DE.xlsx", sheet_name=0)

df.head()

,BLS Code,Lebensmittelbezeichnung,Food name,ENERCJ Energie (Kilojoule) [kJ/100g],ENERCJ Datenherkunft,ENERCJ Referenz,ENERCC Energie (Kilokalorien) [kcal/100g],ENERCC Datenherkunft,ENERCC Referenz,WATER Wasser [g/100g],...,TYR Tyrosin [g/100g],TYR Datenherkunft,TYR Referenz,VAL Valin [g/100g],VAL Datenherkunft,VAL Referenz,"NT Stickstoff, gesamt [g/100g]",NT Datenherkunft,NT Referenz,Hinweis
0,C131000,"Hafer ganzes Korn, roh","Oat whole grain, raw",1443,Formelberechnung,-,343,Formelberechnung,-,11.45,...,0.424,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",0.6425,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",1.724,Aggregation,"Converted value from: Biel, W., et al.; Chemic...",NaN
1,C133000,Hafer Flocken,Oat flakes,1465,Formelberechnung,-,348,Formelberechnung,-,10.07,...,0.57,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",0.81,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",2.115,Analyse,-,NaN
2,C141000,"Gerste ganzes Korn, roh","Barley whole grain, raw",1403,Formelberechnung,-,332,Formelberechnung,-,12.7,...,0.341,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",0.5965,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",1.72,Literatur,Converted value from: Koivistoinen P; Mineral ...,NaN
3,C243000,Gerste Mehl,Barley flour,1434,Formelberechnung,-,340,Formelberechnung,-,6.06,...,0.301,Nährstoffdatenbank,"US Department of Agriculture, Agricultural Res...",0.515,Nährstoffdatenbank,"US Department of Agriculture, Agricultural Res...",1.6,Aggregation,"U.S. Department of Agriculture (USDA), Agricul...",NaN
4,C341000,Mais roh,Maize raw,1418,Formelberechnung,-,336,Formelberechnung,-,11.192,...,0.3935,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",0.4535,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",1.385,Nährstoffdatenbank,"Converted value from: Kirchhoff, E; Souci - Fa...",NaN


In [5]:
def get_instances_by_name(name):
  return df[df['Lebensmittelbezeichnung'].str.contains(name, case=False, na=False)]

get_instances_by_name("hafer flocken")

,BLS Code,Lebensmittelbezeichnung,Food name,ENERCJ Energie (Kilojoule) [kJ/100g],ENERCJ Datenherkunft,ENERCJ Referenz,ENERCC Energie (Kilokalorien) [kcal/100g],ENERCC Datenherkunft,ENERCC Referenz,WATER Wasser [g/100g],...,TYR Tyrosin [g/100g],TYR Datenherkunft,TYR Referenz,VAL Valin [g/100g],VAL Datenherkunft,VAL Referenz,"NT Stickstoff, gesamt [g/100g]",NT Datenherkunft,NT Referenz,Hinweis
1,C133000,Hafer Flocken,Oat flakes,1465,Formelberechnung,-,348,Formelberechnung,-,10.07,...,0.57,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",0.81,Nährstoffdatenbank,"Kirchhoff, E; Souci - Fachmann - Kraut - Die Z...",2.115,Analyse,-,NaN
3274,C133032,"Hafer Flocken, gekocht","Oat flakes, boiled",276,Formelberechnung,-,66,Formelberechnung,-,79.9,...,0.108,Rezeptberechnung,-,0.153,Rezeptberechnung,-,0.4,Rezeptberechnung,-,NaN


In [23]:
def compute_aa_score(row):
  protein = row["PROT625 Protein (Nx6,25) [g/100g]"]

  if protein == 0 or pd.isna(protein):
      return np.nan, None   # ALWAYS two values

  unit_convert_divisor = protein / 1000 # The value needs to be converted into mg per g Protein
  scores = {
        "Isoleucin": row["ILE Isoleucin [g/100g]"] / unit_convert_divisor / aa_ref["Isoleucin"],
        "Leucin": row["LEU Leucin [g/100g]"] / unit_convert_divisor / aa_ref["Leucin"],
        "Lysin": row["LYS Lysin [g/100g]"] / unit_convert_divisor / aa_ref["Lysin"],
        "Methionin + Cystein": row["MET Methionin [g/100g]"] / unit_convert_divisor / aa_ref["Methionin + Cystein"],
        "Phenylalanin + Tyrosin": row["PHE Phenylalanin [g/100g]"] / unit_convert_divisor / aa_ref["Phenylalanin + Tyrosin"],
        "Threonin": row["THR Threonin [g/100g]"] / unit_convert_divisor / aa_ref["Threonin"],
        "Tryptophan": row["TRP Tryptophan [g/100g]"] / unit_convert_divisor / aa_ref["Tryptophan"],
        "Valin": row["VAL Valin [g/100g]"] / unit_convert_divisor / aa_ref["Valin"],
        "Histidin": row["HIS Histidin [g/100g]"] / unit_convert_divisor / aa_ref["Histidin"],
  }


  aa_name, min_score = min(scores.items(), key=lambda x: x[1])
  return scores["Isoleucin"], scores["Leucin"], scores["Lysin"], scores["Methionin + Cystein"], scores["Phenylalanin + Tyrosin"], scores["Threonin"], scores["Tryptophan"], scores["Valin"], scores["Histidin"]
  


# The function returns the approximated biological values for all instances that contains the given name as substring
def get_aa_scores(name):
  aa_columns = [
    "PROT625 Protein (Nx6,25) [g/100g]",
    "HIS Histidin [g/100g]",
    "ILE Isoleucin [g/100g]",
    "LEU Leucin [g/100g]",
    "LYS Lysin [g/100g]",
    "MET Methionin [g/100g]",
    "PHE Phenylalanin [g/100g]",
    "THR Threonin [g/100g]",
    "TRP Tryptophan [g/100g]",
    "VAL Valin [g/100g]",
    "CYSTE Cystein [g/100g]"
  ]
  products = get_instances_by_name(name)[[
    "Lebensmittelbezeichnung"] + aa_columns]

  if products.empty:
    print("No products found!")
    products["Biologische Wertigkeit"] = np.nan
    products["Limitierende Aminosäure"] = None
    return products[[
        "Lebensmittelbezeichnung",
        "Biologische Wertigkeit",
        "Limitierende Aminosäure"
    ]]

  products[aa_columns] = products[aa_columns].apply(
    pd.to_numeric,
    errors="coerce"
  )
  products[list(aa_ref.keys())] = (
    products.apply(
        compute_aa_score,
        axis=1,
        result_type="expand"
    )
  )
  return products[["Lebensmittelbezeichnung", "PROT625 Protein (Nx6,25) [g/100g]"] + list(aa_ref.keys())]

get_aa_scores("Vollmilch ")

,Lebensmittelbezeichnung,"PROT625 Protein (Nx6,25) [g/100g]",Histidin,Isoleucin,Leucin,Lysin,Methionin + Cystein,Phenylalanin + Tyrosin,Threonin,Tryptophan,Valin
338,"Vollmilch frisch, 3,5 % Fett, pasteurisiert",3.55,1.596244,1.480067,1.815336,1.024328,1.186064,1.714636,2.300469,1.444565,1.690141
341,"H-Vollmilch 3,5 % Fett, ultrahocherhitzt",3.18,1.886792,1.812174,1.956674,1.286449,1.406819,2.050861,2.515723,1.773907,2.096436
683,"Vollmilch 3,5 % Fett, laktosefrei, pasteurisiert",3.55,1.596244,1.480067,1.815336,1.024328,1.186064,1.714636,2.347418,1.444565,1.690141
689,"H-Vollmilch 3,5 % Fett, laktosefrei, ultrahoch...",3.18,1.886792,1.812174,1.956674,1.286449,1.406819,2.050861,2.620545,1.773907,2.096436


In [32]:
def combine_products(product_names):
  milch = get_aa_scores("Vollmilch ")
  weizen = get_aa_scores("Weizen Mehl")
  n = min(len(milch), len(weizen))
  combination = (milch[list(aa_ref.keys())].iloc[:n].values + weizen[list(aa_ref.keys())].iloc[:n].values) / 2
  return combination

combine_products("Hafer flocken")

array([[1.34742539, 1.27820989, 1.15775755, 0.87755413, 1.20650285,
        1.45147412, 2.02108147, 1.22727046, 1.59533838],
       [1.50404862, 1.45680465, 1.25866303, 0.99073652, 1.31704217,
        1.6403755 , 2.14980863, 1.40643933, 1.81274402],
       [1.34908625, 1.27115356, 1.22141173, 0.85651651, 1.17299418,
        1.47116379, 2.09198257, 1.24322612, 1.62560302],
       [1.53294371, 1.48942281, 1.21203132, 1.034325  , 1.34495118,
        1.6281408 , 2.1866269 , 1.41399201, 1.78116895]])